# Spark Streaming & Classification

In this lesson we will see the spark streaming module capability and classify our churn dataset.

## Summary
- <a href='#1'>1. Context and Motivation</a>
- <a href='#2'>2. Spark Streaming Overview</a>
    - <a href='#2.1'>2.1. What is stream processing</a>
    - <a href='#2.2'>2.2. Dataset</a>
    - <a href='#2.3'>2.3. Streaming Example</a>
    - <a href='#2.3.1'>2.3.1 Sorting and Filtering</a>
    - <a href='#2.3.2'>2.3.2 Aggregations</a>
- <a href='#3'>3.  Exercises</a>
    - <a href='#3.1'>3.1. Feature Transformation</a>
    - <a href='#3.2'>3.2. EDA</a>
    - <a href='#3.3'>3.3. Classification</a>
        - <a href='#3.3.1'>3.3.1 Logistic Regression</a>
        - <a href='#3.3.2'>3.3.2 (SVM)Support vector Machine</a>
        - <a href='#3.3.3'>3.3.3 Decision Trees</a>
        - <a href='#3.3.4'>3.3.4 Feature Importance</a>
    - <a href='#3.4'>3.4. Evaluation</a>
        - <a href='#3.4.1'>3.4.1 Confusion Matrix</a>
        - <a href='#3.4.2'>3.4.2 Accuracy</a>
        - <a href='#3.4.3'>3.4.3 Precision</a>
        - <a href='#3.4.4'>3.4.4 Recall</a>
        - <a href='#3.4.6'>3.4.6 AUC(Area Under the Roc Curve)</a>
- <a href='#4'>4.  References</a>

# <a id='1'>1. Context and Motivation</a>

Spark Streaming allow us to use realtime stream processing in applications such:

* **Realtime Reporting**  

* **Incremental ETL**

* **Real-time decision making**

* **Online machine learning**


## <a id='2.1'>2.1. What is stream processing</a>

Structured Streaming enables users build **continuous applications**. A **continous application** is an end-to-end application that reacts to data in real time by combining a variety of tools: streaming jobs, batch jobs, joins between streaming and offline data, and interactive ad-hoc queries.

<img src="Spark_streaming_output.png" width="500px"/>

## <a id='2.2'>2.2. Dataset</a>

Heterogeneity Human Activity Recognition Dataset.  
The data consists of smartphone and **smartwatch sensor readings** from a variety of devices
specifically, the **accelerometer and gyroscope**, sampled at the highest possible frequency
supported by the devices.    
Readings from these sensors were recorded while users performed
activities like biking, sitting, standing, walking, and so on. There are several different
smartphones and smartwatches used, and nine total users.

Put the data in **hdfs**: `hdfs dfs -put /activity_data/*`

See https://hadoop.apache.org/docs/r2.4.1/hadoop-project-dist/hadoop-common/FileSystemShell.html#put

In [ ]:
static = spark.read.json("activity-data/") # read the dataframe as static to get the schema 
dataSchema = static.schema

In [ ]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json("activity-data/")

## <a id='2.3'>2.3. Streaming Example</a>

In [ ]:
activityCounts = streaming.groupBy("gt").count()# activity being performed by the user at that point in time

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 5) # avoid too many shuffle partitions

Output Modes: 
    
* Append (only add new records to the output sink)
* Update (update changed records in place)
* Complete (rewrite the full output)

In [ ]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start() # in-memory table and mode can be complete

In [ ]:
#activityQuery.awaitTermination() # prevent the driver process from exiting while the query is active.

In [ ]:
spark.streams.active

In [ ]:
spark.sql("SELECT * FROM activity_counts").show(5)

### <a id='2.3.1'>2.3.1. Selection and Filtering</a>

In [ ]:
from pyspark.sql.functions import expr
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time")\
.writeStream\
.queryName("simple_transform")\
.format("memory")\
.outputMode("append")\
.start()

In [ ]:
spark.sql("SELECT * FROM simple_transform").show(5)

### <a id='2.3.2'>2.3.2. Aggregations</a>

In [ ]:
deviceModelStats = streaming.cube("gt", "model").avg()\
.drop("avg(Arrival_time)")\
.drop("avg(Creation_Time)")\
.drop("avg(Index)")\
.writeStream.queryName("device_counts").format("memory")\
.outputMode("complete")\
.start()# cube, on the phone model and activity and the average x, y, z accelerations of our sensor

In [ ]:
spark.sql("SELECT * FROM device_counts").show(5)

# <a id='3'>3. Exercises</a>

# Customer Churn

Customer churn, also known as customer attrition, customer turnover, or customer defection, is the loss of clients or customers.

Telephone service companies, Internet service providers, pay TV companies, insurance firms, and alarm monitoring services, often use customer churn analysis and customer churn rates as one of their key business metrics because the cost of retaining an existing customer is far less than acquiring a new one. Companies from these sectors often have customer service branches which attempt to win back defecting clients, because recovered long-term customers can be worth much more to a company than newly recruited clients.

Companies usually make a distinction between voluntary churn and involuntary churn. Voluntary churn occurs due to a decision by the customer to switch to another company or service provider, involuntary churn occurs due to circumstances such as a customer's relocation to a long-term care facility, death, or the relocation to a distant location. In most applications, involuntary reasons for churn are excluded from the analytical models. Analysts tend to concentrate on voluntary churn, because it typically occurs due to factors of the company-customer relationship which companies control, such as how billing interactions are handled or how after-sales help is provided.

Predictive analytics use churn prediction models that predict customer churn by assessing their propensity of risk to churn. Since these models generate a small prioritized list of potential defectors, they are effective at focusing customer retention marketing programs on the subset of the customer base who are most vulnerable to churn.

## Column Description   

| Column     | Type       | Description |
|--------  |---------  |: --------- |
| **customerID** | String | Customer ID |
| **gender** | String | Whether the customer is a male or a female |
| **SeniorCitizen** | Integer | Whether the customer is a senior citizen or not (1, 0) |
| **Partner** | String | Whether the customer has a partner or not (Yes, No) |
| **Dependents** | String | Whether the customer has dependents or not (Yes, No) |
| **tenure** | Integer | Number of months the customer has stayed with the company |
| **PhoneService** | String | Whether the customer has a phone service or not (Yes, No) |
| **MultipleLines** | String | Whether the customer has multiple lines or not (Yes, No, No phone service) |
| **InternetService** | String | Customer’s internet service provider (DSL, Fiber optic, No) |
| **OnlineSecurity** | String | Whether the customer has online security or not (Yes, No, No internet service) |
| **OnlineBackup** | String | Whether the customer has online backup or not (Yes, No, No internet service) |
| **DeviceProtection** | String | Whether the customer has device protection or not (Yes, No, No internet service) |
| **TechSupport** | String | Whether the customer has tech support or not (Yes, No, No internet service) |
| **StreamingTV** | String | Whether the customer has streaming movies or not (Yes, No, No internet service) |
| **StreamingMovies** | String | Whether the customer has a partner or not (Yes, No) |
| **Contract** | String | The contract term of the customer (Month-to-month, One year, Two year) |
| **PaperlessBilling** | String | Whether the customer has paperless billing or not (Yes, No) |
| **PaymentMethod** | String | The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)) |
| **MonthlyCharges** | Double | The amount charged to the customer monthly |
| **TotalCharges** | String | The total amount charged to the customer |
| **Churn** | String | Whether the customer churned or not (Yes or No) |

## <a id='3.1'>3.1. Feature Transformation</a>

In [ ]:
df = spark.read.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import when  

In [ ]:
# check null values
df.where(col("TotalCharges").isNull()).count() 

In [ ]:
# Remove null values
df = df.na.drop(subset=["TotalCharges"]) ## Drop null values

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
encoder = OneHotEncoder()\
.setInputCol("gender")\
.setOutputCol("gender").show()

In [ ]:
#encoder.transform(df.select("gender")).show()

stringIndexer = StringIndexer(inputCol="gender", outputCol="genderIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
#indexed.show()
#encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
#encoded = encoder.transform(indexed)
#encoded.show()
indexed.select("genderIndex").show()

In [ ]:
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    df  = df.select(i).replace(["No internet service"], ["No"], i)


In [ ]:
from pyspark.sql.functions import regexp_replace, col
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols :
    df = df.withColumn(i, regexp_replace(i, "No internet service", "No"))

In [ ]:
df.select("OnlineBackup").show()

In [ ]:
df.select("gender").show()

In [ ]:
df = df.withColumn('label', when(df["Churn"] == "Yes" , 1).otherwise(0)) # convert into 0 or 1

In [ ]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="label ~ . + Churn:TotalCharges + Churn:MonthlyCharges + Churn:SeniorCitizen")

In [ ]:
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(1)

In [ ]:
train, test = preparedDF.randomSplit([0.7, 0.3]) ## preparing dataframe

## <a id='3.2'>3.2. EDA</a>

## <a id='3.3'>3.3. Classification</a>

### <a id='3.3.1'>3.3.1 Logistic Regression</a>


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label",featuresCol="features",  regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(train)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

### <a id='3.3.2'>3.3.2 (SVM)Support vector Machine</a> 

**Create a model with support vector machine algoritm**   
See https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine

### <a id='3.3.3'>3.3.3 Decision Trees</a>
**Create a model with DecisionTreeClassifier algoritm**   
See https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine

### <a id='3.3.4'>3.3.4 Feature importance</a>
**Using the decision tree algorithm plot the tree and see what are the most important features that define the model.**

## <a id='3.4'>3.4 Evaluation</a>

In [ ]:
trainingSummary = lrModel.summary # model summary to get the metrics

## <a id='3.4.1'>3.4.1. Confusion Matrix</a>

<img src="confusion_matrix.png" width="350px"/>

* True Positive: -> Interpretation: You predicted positive and it’s true.
* True Negative: -> Interpretation: You predicted negative and it’s true.
* False Positive: -> Interpretation: You predicted positive and it’s false.
* False Negative: -> Interpretation: You predicted negative and it’s false.

## <a id='3.4.2'>3.4.2. Accuracy</a>

**Accuracy = (TP+TN)/(TP+FP+FN+TN)** -> How many churners did we correctly label out of all the churners

In [ ]:
accuracy = trainingSummary.accuracy

In [ ]:
print("Accuracy:"+ str(accuracy))

In [ ]:
# What are the accuracy for the other modules??

## <a id='3.4.3'>3.4.3. Precision</a>

**Precision = TP/(TP+FP)**  -> How many of those who we labeled as churners are actually churners

In [ ]:
precision = trainingSummary.weightedPrecision

In [ ]:
print("Precision:"+ str(precision))

In [ ]:
# What are the precision for the other modules??

## <a id='3.4.4'>3.4.4. Recall</a>

**Recall = TP/(TP+FN)** -> Of all the people who are churners, how many of those we correctly predict

In [ ]:
recall = trainingSummary.weightedRecall

In [ ]:
print("Recall:"+ str(precision))

In [ ]:
# What are the Recall for the other modules??

## <a id='3.4.4'>3.4.4. AUC(Area Under the Roc Curve)</a>

See https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5

In [ ]:
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

In [ ]:
trainingSummary.roc.show()

## <a id='4'>4. References</a>

https://github.com/databricks/Spark-The-Definitive-Guide

https://en.wikipedia.org/wiki/Confusion_matrix